# 13.1 pandas와 모델 코드의 인터페이스

In [1]:
# 모델 개발의 일반적인 흐름은 데이터를 불로오고 정제하는 과정은 pandas를 이용,
# 그 후 모델 개발을 위해 모델링 라이브러리로 넘어가게 된다.
# 모델을 개발하는 과정에서 중요한 단계는 특징을 선택하고 추출하는 피처 엔지니어링 (데이터 요약이나 GroupBy도구)

import pandas as pd
import numpy as np
from pandas import Series, DataFrame

In [2]:
data=pd.DataFrame({
    'x0':[1,2,3,4,5],
    'x1':[0.01,-0.01,0.25,-4.1,0.],
    'y':[-1.5,0.,3.6,1.3,-2.]})
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [3]:
data.columns

Index(['x0', 'x1', 'y'], dtype='object')

In [4]:
data.values

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

In [5]:
# 2차원 ndarray와 컬럼 이름 리스트(필요하다면)을 넘겨서 DF로 되돌림

df2=pd.DataFrame(data.values, columns=['one','two','three']) 
df2

,one,two,three
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


In [8]:
# .values속성은 데이터가 한가지 타입으로 이뤄졌다는 가정 하에 사용.
# 만약 데이터 속성이 한가지가 아니라면 파이썬 객체의 ndarray반환

df3=data.copy()

In [9]:
df3['strings'] = ['a','b','c','d','e']
df3

,x0,x1,y,strings
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,c
3,4,-4.10,1.3,d
4,5,0.00,-2.0,e


In [10]:
df3.values

array([[1, 0.01, -1.5, 'a'],
       [2, -0.01, 0.0, 'b'],
       [3, 0.25, 3.6, 'c'],
       [4, -4.1, 1.3, 'd'],
       [5, 0.0, -2.0, 'e']], dtype=object)

In [11]:
# 전체 컬럼 중 일부만 사용하고 싶은 경우: loc를 이용하여 values속성에 접근

model_cols=['x0','x1']

In [12]:
data.loc[:, model_cols].values

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

In [13]:
# 예) 데이터셋에 숫자가 아닌 컬럼이 있다고 가정

data['category']=pd.Categorical(['a','b','a','a','b'],
                               categories=['a','b'])
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


In [14]:
dummies=pd.get_dummies(data.category, prefix='category') 
# 'category'컬럼을 더미값으로 치환하고 싶다면 더미값 생성 후 'category'컬럼을 삭제한 다음 결과와 합치기

In [15]:
data_with_dummies= data.drop('category',axis=1).join(dummies)
data_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,0.25,3.6,1,0
3,4,-4.10,1.3,1,0
4,5,0.00,-2.0,0,1


# 13.2 Patsy를 이용해서 모델 생성

In [17]:
# patsy는 통계 모델(선형 모델)을 위한 파이썬 라이브러리
# a + b 문법은 더하기의 의미가 아닌 모델을 위해 생성된 배열을 설계하는 용법
# patsy.dmatrices함수는 수식 문자열과 데이터셋을 함께 받아 선형 모델을 위한 설계 배열을 만들어 낸다.

import patsy

In [18]:
data=pd.DataFrame({
    'x0':[1,2,3,4,5],
    'x1':[0.01,-0.01,0.25,-4.1,0.],
    'y':[-1.5,0.,3.6,1.3,-2]
})
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [19]:
y, X = patsy.dmatrices('y ~ x0 + x1', data)

In [20]:
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [21]:
X # Intercept는 최소자승회귀와 같은 선형 모델을 위한 표현

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

In [22]:
# patsy의 DesignMatrix 인스턴스는 몇가지 추가 데이터가 포함된 numpy ndarray로 볼 수 있다.

np.asarray(y) 

array([[-1.5],
       [ 0. ],
       [ 3.6],
       [ 1.3],
       [-2. ]])

In [23]:
np.asarray(X)

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

In [24]:
patsy.dmatrices('y ~ x0 + x1 + 0',data)[1] # 모델에 0을 더해서 intercept(절편) 제거

DesignMatrix with shape (5, 2)
  x0     x1
   1   0.01
   2  -0.01
   3   0.25
   4  -4.10
   5   0.00
  Terms:
    'x0' (column 0)
    'x1' (column 1)

In [31]:
coef, resid, _, _ =np.linalg.lstsq(X,y, rcond=None) 
# patsy객체는 최소자승회귀분석을 위해 numpy.linalg.lstsq같은 알고리즘에 바로 넘길 수 있다.

In [32]:
coef 

array([[ 0.31290976],
       [-0.07910564],
       [-0.26546384]])

In [33]:
coef = pd.Series(coef.squeeze(), index=X.design_info.column_names) # 모델 메타데이터는 design_info 속성을 통해 얻음
coef

Intercept    0.312910
x0          -0.079106
x1          -0.265464
dtype: float64

* Patsy 용법으로 데이터 변환하기

In [34]:
# 파이썬 코드를 Patsy용법과 섞어서 사용하면 Patsy문법을 해석하는 과정에서 해당 함수를 찾아 실행

y, X =patsy.dmatrices('y ~ x0 + np.log(np.abs(x1)+1)', data)
X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

In [35]:
y, X =patsy.dmatrices('y ~ standardize(x0) + center(x1)', data) # 표준화(평균0, 분산1)와 센터링(평균값을 뺌)
X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

In [36]:
# 모델링 과정에서 모델을 어떤 데이터셋에 피팅한 후 다른 모델에 기반하여 평가하는 경우- 홀드아웃이거나 신규 데이터가 나중에 관측
# 센터링이나 표준화를 새로운 데이터에 기반하여 적용하는 경우- 상태를 가지는 변환이라 칭함, 주의 요망

new_data=pd.DataFrame({
    'x0':[6,7,8,9],
    'x1':[3.1,-0.5,0,2.3],
    'y':[1,2,3,4]
})

In [37]:
new_X=patsy.build_design_matrices([X.design_info], new_data) 
# 입력할 원본 데이터셋에서 저장한 정보를 사용하여 출력 데이터를 만들어낼 때 적용
new_X

[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        3.87
           1          2.82843        0.27
           1          3.53553        0.77
           1          4.24264        3.07
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

In [38]:
y, X =patsy.dmatrices('y ~ I(x0+x1)', data) # 데이터셋에서 이름으로 컬럼을 추가하고 싶다면 I라는 특수한 함수 사용

In [39]:
X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

* 범주형 데이터와 Patsy

In [40]:
# Patsy에서 비산술 용법을 사용하면 기본적으로 더미 변수로 변환
# 만약 intercept가 존재한다면 공선성을 피하기 위해 레벨 하나는 남겨두게 된다.

data=pd.DataFrame({
    'key1':['a','a','b','b','a','b','a','b'],
    'key2':[0,1,0,1,0,1,0,0],
    'v1':[1,2,3,4,5,6,7,8],
    'v2':[-1,0,2.5,-0.5,4.0,-1.2,0.2,-1.7]
})

In [46]:
y, X = patsy.dmatrices('v2 ~ key1', data)

In [47]:
X

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

In [50]:
y, X =patsy.dmatrices('v2 ~ key1 + 0',data) # 모델에서 intercept를 생략하면 각 범주값의 컬럼은 모델 설계 배열에 포함

In [51]:
X

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

In [52]:
y, X = patsy.dmatrices('v2 ~ C(key2)', data) # 산술 컬럼은 C함수를 이용하여 범주형으로 해석

In [53]:
X

DesignMatrix with shape (8, 2)
  Intercept  C(key2)[T.1]
          1             0
          1             1
          1             0
          1             1
          1             0
          1             1
          1             0
          1             0
  Terms:
    'Intercept' (column 0)
    'C(key2)' (column 1)

In [54]:
# 모델에서 여러 범주형 항을 사용한다면 ANOVA(분산분석)모델에서처럼 key1:key2같은 용법을 사용하게 되므로 더 복잡

data['key2']=data['key2'].map({0:'zero', 1:'one'})
data

,key1,key2,v1,v2
0,a,zero,1,-1.0
1,a,one,2,0.0
2,b,zero,3,2.5
3,b,one,4,-0.5
4,a,zero,5,4.0
5,b,one,6,-1.2
6,a,zero,7,0.2
7,b,zero,8,-1.7


In [55]:
y, X=patsy.dmatrices('v2 ~ key1 + key2', data)
X

DesignMatrix with shape (8, 3)
  Intercept  key1[T.b]  key2[T.zero]
          1          0             1
          1          0             0
          1          1             1
          1          1             0
          1          0             1
          1          1             0
          1          0             1
          1          1             1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)

In [57]:
y, X=patsy.dmatrices('v2 ~ key1 + key2 + key1:key2', data)
X

DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zero]  key1[T.b]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

# 13.3 statsmodels 소개

: statsmodels는 다양한 종류의 통계 모델 피팅, 통계 테스트 수행, 데이터 탐색과 시각화를 위한 파이썬 라이브러리

In [58]:
# 선형 모델, 선형복합효과 모델, 아노바 메서드, 시계열 처리 및 상태 공간 모델, 일반적률추정법 포함

* 선형 모델 예측

In [59]:
# statsmodels는 배열 기반과 용법 기반이라는 두가지 주요한 인터페이스를 가진다. API모듈을 임포트하여 사용

import statsmodels.api as sm
import statsmodels.formula.api as smf

In [60]:
# dnorm은 특정 평균과 분산을 가지는 정규분포 데이터를 생성하기 위한 도움 함수
def dnorm(mean, variance, size=1):
    if isinstance(size, int):
        size=size,
    return mean+np.sqrt(variance) * np.random.randn(*size)

# 동일한 난수 발생을 위해 시드값 직접 지정
np.random.seed(12345)

N=100
X=np.c_[dnorm(0, 0.4, size=N),
       dnorm(0, 0.6, size=N),
       dnorm(0, 0.2, size=N)]
eps=dnorm(0, 0.1, size=N)
beta=[0.1,0.3,0.5]

y=np.dot(X, beta) + eps

In [61]:
X[:5]

array([[-0.12946849, -1.21275292,  0.50422488],
       [ 0.30291036, -0.43574176, -0.25417986],
       [-0.32852189, -0.02530153,  0.13835097],
       [-0.35147471, -0.71960511, -0.25821463],
       [ 1.2432688 , -0.37379916, -0.52262905]])

In [62]:
y[:5]

array([ 0.42786349, -0.67348041, -0.09087764, -0.48949442, -0.12894109])

In [63]:
X_model=sm.add_constant(X) # sm.add_constant함수는 intercept컬럼을 기존 행렬에 더할 수 있다.
X_model[:5]                # 선형 모델은 일반적으로 intercept와 함께 피팅됨

array([[ 1.        , -0.12946849, -1.21275292,  0.50422488],
       [ 1.        ,  0.30291036, -0.43574176, -0.25417986],
       [ 1.        , -0.32852189, -0.02530153,  0.13835097],
       [ 1.        , -0.35147471, -0.71960511, -0.25821463],
       [ 1.        ,  1.2432688 , -0.37379916, -0.52262905]])

In [64]:
model = sm.OLS(y,X) # sm.OLS 클래스는 최소자승 선형회귀에 피팅 가능
model

In [65]:
results=model.fit() # 모델의 fit메서드는 예측 모델 인자와 다른 분석 정보를 포함하는 회귀 결과 객체 반환
results.params

array([0.17826108, 0.22303962, 0.50095093])

In [66]:
print(results.summary()) # results의 summary메서드를 호출하여 해당 모델의 자세한 분석 결과를 출력하도록 가능

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.430
Model:                            OLS   Adj. R-squared (uncentered):              0.413
Method:                 Least Squares   F-statistic:                              24.42
Date:                Thu, 25 Feb 2021   Prob (F-statistic):                    7.44e-12
Time:                        02:21:49   Log-Likelihood:                         -34.305
No. Observations:                 100   AIC:                                      74.61
Df Residuals:                      97   BIC:                                      82.42
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [67]:
# 모델 인자가 하나의 DataFrame에 들어있다고 가정

data=pd.DataFrame(X, columns=['col0','col1','col2'])

In [68]:
data['y']=y

In [69]:
data[:5]

,col0,col1,col2,y
0,-0.129468,-1.212753,0.504225,0.427863
1,0.302910,-0.435742,-0.254180,-0.673480
2,-0.328522,-0.025302,0.138351,-0.090878
3,-0.351475,-0.719605,-0.258215,-0.489494
4,1.243269,-0.373799,-0.522629,-0.128941


In [70]:
results=smf.ols('y~col0+col1+col2', data=data).fit() # 이제 statsmodels의 API와 Patsy의 문자열 용법 사용가능

In [71]:
results.params

Intercept    0.033559
col0         0.176149
col1         0.224826
col2         0.514808
dtype: float64

In [72]:
results.tvalues

Intercept    0.952188
col0         3.319754
col1         4.850730
col2         6.303971
dtype: float64

In [73]:
results.predict(data[:5]) # 주어진 새로운 샘플 데이터를 통해 예측 모델 인자에 전달한 예측값 계산 가능

0   -0.002327
1   -0.141904
2    0.041226
3   -0.323070
4   -0.100535
dtype: float64

* 시계열 처리 예측

In [74]:
# 시계열 분석을 위한 모델에는 자동회귀 처리, 칼만 필터링과 다른 상태 공간 모델 그리고 다변 자동회귀 모델 존재
# 자동회귀 구조와 노이즈를 이용해서 시계열 데이터를 시뮬레이션

import random

In [76]:
init_x=4

values=[init_x, init_x]
N=1000

# 이 데이터는 인자가 0.8과 -0.4인 AR(2) 구조(두 개의 지연)다.
b0=0.8
b1=-0.4
noise=dnorm(0,0.1,N)
for i in range(N):
    new_x=values[-1]* b0 + values[-2]* b1 + noise[i]
    values.append(new_x)

In [77]:
MAXLAGS=5 # AR모델을 피팅할 땐 포함시켜야할 지연 항을 얼마나 두어야 하는지 알지 못하므로 적당히 큰 값으로 모델을 피팅

In [80]:
model=sm.tsa.AR(values)

In [81]:
results=model.fit(MAXLAGS)

In [82]:
results.params # 결과에서 예측된 인자는 intercept를 가지고 그 다음에 두 지연에 대한 예측치를 가짐

array([ 0.00791554,  0.71074082, -0.31689896, -0.06724519,  0.0041215 ,
       -0.00079061])

# 13.4 scikit-learn 소개

: 범용 파이썬 머신러닝 툴킷

In [83]:
# 지도학습과 비지도학습 메서드 포함
# 모델 선택, 평가, 데이터 변형, 데이터 적재, 모델 유지 및 기타 작업을 위한 도구들을 제공

In [85]:
train= pd.read_csv('datasets/titanic/train.csv')
test=pd.read_csv('datasets/titanic/test.csv')

In [86]:
train[:4]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S


In [87]:
train.isnull().sum() # statsmodels나 scikot-learn라이브러리는 누락된 데이터 처리를 못하므로 데이터셋에 빠진 값이 있는지 살펴봄

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [88]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [90]:
# 나이를 기반으로 생존 여부를 예측하고자 하지만 누락된 데이터가 존재
# 결측치를 보완하기 위해 간단히 학습 데이터셋의 중간값을 채워 넣는 방법으로 처리

impute_value = train['Age'].median()

In [92]:
train['Age'] = train['Age'].fillna(impute_value)
test['Age']=test['Age'].fillna(impute_value)

In [93]:
# 모델을 명세하기 위해 IsFemale컬럼을 추가해서 'Sex'컬럼을 인코딩한다.
train['IsFemale'] = (train['Sex']=='female').astype(int)
test['IsFemale'] = (test['Sex']=='female').astype(int)

In [94]:
predictors=['Pclass','IsFemale','Age'] 
X_train=train[predictors].values
X_test=test[predictors].values

In [95]:
y_train = train['Survived'].values

In [96]:
X_train[:5]

array([[ 3.,  0., 22.],
       [ 1.,  1., 38.],
       [ 3.,  1., 26.],
       [ 1.,  1., 35.],
       [ 3.,  0., 35.]])

In [97]:
y_train[:5]

array([0, 1, 1, 1, 0], dtype=int64)

In [98]:
# scikit-learn의 LogisticRegression 모델을 이용해서 model인스턴스 생성

from sklearn.linear_model import LogisticRegression

In [99]:
model=LogisticRegression()

In [100]:
model.fit(X_train, y_train) # statsmodels와 유사하게 model의 fit메서드를 이용하여 이 모델을 학습 데이터에 피팅 가능

LogisticRegression()

In [101]:
y_predict=model.predict(X_test) # model.predict를 이용하여 테스트 데이터셋으로 예측 가능
y_predict[:10]

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0], dtype=int64)

In [ ]:
# 테스트 데이터셋의 실제 생존 여부 값을 가지고 있다면 정확도나 기타 오류율을 계산 가능
(y_true == y_predict).mean()

In [102]:
# 많은 모델은 조절할 수 있는 인자를 가지고 학습 데이터에 오버피팅되는 것을 피하기 위해 교차검증 기법 활용
# 교차검증은 학습 데이터를 분할하여 예측을 위한 샘플로 활용하는 방식으로 작동
# 모델 정확도 점수(ex.평균 제곱오차)에 기반하여 모델 인자에 대한 그리드 검색 수행

In [103]:
# 로지스틱 회귀 같은 모델에서는 교차검증을 내장한 추정 클래스 제공
# 예) LogisticRegressionCV클래스는 모델 정규화 인자 C에 대한 그리드 검색을 얼마나 정밀히 수행할 것인지 나타내는 인자와 함께 사용

from sklearn.linear_model import LogisticRegressionCV

In [104]:
model_cv = LogisticRegressionCV(10)

C:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass Cs=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [105]:
model_cv.fit(X_train, y_train)

LogisticRegressionCV()

In [106]:
# 직접 교차검증을 수행하려면 데이터를 분할하는 과정을 도와주는 cross_val_score함수를 이용
# 예) 학습 데이터를 겹치지 않는 4개의 그룹으로 나누려면 아래와 같이 함

from sklearn.model_selection import cross_val_score

In [107]:
model = LogisticRegression(C=10)

In [109]:
scores=cross_val_score(model, X_train, y_train, cv=4)
scores

array([0.77578475, 0.79820628, 0.77578475, 0.78828829])